In [45]:
import pandas as pd

In [46]:
df_classifier_test = pd.read_csv('data/submissions/classifier_test.csv').fillna('')

In [47]:
df_classifier_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [48]:
df_classifier_test_confidence = pd.read_csv('data/classifier_test_confidence.csv').fillna('')

In [49]:
df_classifier_test_confidence.head()

,ImageId_ClassId,Confidence
0,004f40c73.jpg,0.016510
1,006f39c41.jpg,0.045337
2,00b7fb703.jpg,0.176907
3,00bbcd9af.jpg,0.001386
4,0108ce457.jpg,0.277736


In [50]:
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence<0.05]))
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence>0.8]))

1404
196


In [51]:
df_classifier_confident = df_classifier_test_confidence[(df_classifier_test_confidence.Confidence<0.05) | 
                                                        (df_classifier_test_confidence.Confidence>0.8)]

In [52]:
df_classifier_confident = df_classifier_confident.reset_index(drop=True)

In [53]:
df_classifier_confident.to_csv('data/confident_classification.csv', index=False)

In [54]:
#df_segmentation_test_confidence = pd.read_csv('data/segmentation_test_confidence.csv').fillna('')

In [55]:
#df_segmentation_test_confidence.head()

In [59]:
df_segmentation_test = pd.read_csv('data/submissions/segmentation_test.csv').fillna('')

In [60]:
df_segmentation_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [61]:
df_train = pd.read_csv('data/train.csv').fillna('')

In [62]:
df_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [63]:
df_classifier_confident

,ImageId_ClassId,Confidence
0,004f40c73.jpg,0.016510
1,006f39c41.jpg,0.045337
2,00bbcd9af.jpg,0.001386
3,0109b68ec.jpg,0.000571
4,010ec96b4.jpg,0.019511
...,...,...
1595,ff6aa9608.jpg,0.998115
1596,ff6b5bf9d.jpg,0.000389
1597,ffaf13d96.jpg,0.000044
1598,ffc9a6187.jpg,0.000676


In [64]:
df_pseudo_train

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,
...,...,...
57471,ffc9a6187.jpg_4,
57472,ffdb60677.jpg_1,
57473,ffdb60677.jpg_2,
57474,ffdb60677.jpg_3,


In [81]:
pseudolabel = []

for ind, row in df_classifier_confident.iterrows():
    if row['Confidence'] < 0.05 or row['Confidence'] > 0.8:
        for i in range(4):
            class_id = row.ImageId_ClassId + '_' + str(i+1)

            rle = df_segmentation_test.loc[df_segmentation_test.ImageId_ClassId == class_id].EncodedPixels.values
            if len(rle) > 0:
                rle = rle[0]
            else:
                rle = None
            pseudolabel.append({
                'ImageId_ClassId': class_id,
                'EncodedPixels': rle,
            })

In [88]:
df_pseudolabel_test = pd.DataFrame(pseudolabel)

In [89]:
df_pseudolabel_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [90]:
len(df_pseudolabel_test), len(df_train)

(6400, 50272)

In [96]:
df_pseudo_train = df_train.append(df_pseudolabel_test, ignore_index=True)

In [97]:
df_pseudo_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [98]:
len(df_pseudo_train), len(df_train), len(df_pseudolabel_test)

(56672, 50272, 6400)

In [101]:
df_pseudo_train.to_csv('data/train_pseudolabel.csv', index=False)